In [125]:
#Libraries to needed for the code
from scipy import signal 
import numpy as np
import timeit

import arlpy.uwapm as pm
import arlpy.plot as aplt

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from bokeh.models.annotations import Title
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Category10,viridis,Spectral11
from bokeh.models import ColumnDataSource, Range1d, BoxAnnotation, HoverTool, LogColorMapper, LogTicker, ColorBar
output_notebook()

def impulse_response(h,r,d1,d2,samplefreq, fc, **kwargs):
    Type = kwargs.get('Type', 'valid')

    ssp = 1500
    
    surface = np.array([[r, 0.5+0.5*np.sin(2*np.pi*0.05*r)] for r in np.linspace(0,1000,1001)])
    maxAngle = 85 #180*(np.arctan(h/(r/2)))/np.pi 
    env = pm.create_env2d(depth = h, rx_range = r, tx_depth = d1, rx_depth = d2, frequency = fc, max_angle = maxAngle, min_angle = -maxAngle, surface = surface, soundspeed = ssp)
    arrivals = pm.compute_arrivals(env)
    Hn = pm.arrivals_to_impulse_response(arrivals,samplefreq)
    
    if Type == 'valid':
        return Hn.real
    if Type == 'full':
        return env, arrivals, Hn.real
    

def multiFrequencyResponse(h,r,d1,d2_list,samplefreq, fc, bw, **kwargs):

    bandwidth = kwargs.get('bwe', True)
    lowfreq = fc-(bw/2.0)
    highfreq = fc+(bw/2.0)
    print("lowfreq = ", lowfreq)
    print("highfreq = ", highfreq)
    freq_final_list_trunc = []
    freq_final_list = []
    max_list_of_new_list_trunc = []
    max_list_of_new_list = []
    top_max,top_freq = 0,0
    #low_max, low_freq = 0,0
    
    for d2 in d2_list:
        Hn = impulse_response(h,r,d1,d2,samplefreq,fc)
        max_new_list = np.abs(np.fft.rfft(Hn), 5000)
        signalSize = len(Hn)
        print("len of Hn: ", signalSize)
        freq_list_ori = np.fft.fftfreq(signalSize,1000/samplefreq)
        count_low, count_high, count_high2 = 0,0,0
        for i in range(len(freq_list_ori)):
            if freq_list_ori[i] <= lowfreq or freq_list_ori[i] <=0:
                count_low += 1
            if freq_list_ori[i] >= highfreq or freq_list_ori[i] <=0:
                count_high += 1
            if freq_list_ori[i] <=0:
                count_high2 += 1
        print("count_low = ", count_low)
        print("len(freq_list_ori) = ", len(freq_list_ori))
        print("len(max_new_list) = ", len(max_new_list))
        freq_list = freq_list_ori[0:len(freq_list_ori)-count_high2]
        freq_final_list.append(freq_list)
        max_list_of_new_list.append(max_new_list)
        
        top_max = len(max_new_list)-count_high
        top_freq = len(freq_list_ori)-count_high
        
#        if f_top_max <= top_max:
#            top_max = f_top_max
        
#        if f_top_freq <= top_freq:
#            top_freq = f_top_freq
        
#    for d2 in d2_list:
#        Hn = impulse_response(h,r,d1,d2,samplefreq,fc)
#        max_new_list = np.abs(np.fft.rfft(Hn))
#        print(count_high)
        count_low_max = int(count_low/2)
        print("count_low_max = ", count_low_max)
        max_new_list_trunc = max_new_list[count_low_max:]
        freq_list_trunc = freq_list[count_low:top_freq]
        max_list_of_new_list_trunc.append(max_new_list_trunc)
        freq_final_list_trunc.append(freq_list_ori)
        
       
    if bandwidth == True:
        return freq_final_list_trunc, max_list_of_new_list_trunc
    else:
        return freq_final_list, max_list_of_new_list


Loading BokehJS ...

In [113]:
samplefreq = 100e3 #Hz
h, r, d1, fc, bw = 100,100, 10, 20, 20
d2_list = np.linspace(1,5,4)

In [126]:
freq_list, freq_response = multiFrequencyResponse(h,r,d1,d2_list,samplefreq,fc,bw, bwe = False)

roundD2_test = ['%.2f'% elem for elem in d2_list]
data = {'xs': freq_list,
        'ys': freq_response,
        'labels': roundD2_test,
        'mypalette': viridis(len(d2_list))}

source = ColumnDataSource(data)

FR = figure(title = "Frequency Response of the Channel of range {0} m".format(r), x_axis_label = "Frequency(kHz)", y_axis_label = "Frequency Response",plot_width=950, plot_height=400,logo = None)
FR.title.align = "center"
FR.title.text_color = "black"
FR.title.text_font_size = "15px"

FR.multi_line(xs='xs', ys='ys', legend='labels', line_color = 'mypalette',source=source)
show(FR)

lowfreq =  10.0
highfreq =  30.0


TypeError: return arrays must be of ArrayType

In [32]:
#printing 2D from above
X,Y = np.meshgrid(freq_list, d2_list)
Z = freq_response

fig, ax = plt.subplots(figsize=(9, 8))

p = ax.pcolor(X, Y, Z, cmap=plt.cm.Greys, vmin=np.abs(Z).min(), vmax=np.abs(Z).max())
cb = fig.colorbar(p)
plt.title ("Frequency Response @ different receiver depth")
plt.xlabel("frequency (kHz)")
plt.ylabel('Receiver Depth')
plt.show(cb)

ValueError: operands could not be broadcast together with shapes (14829,) (14877,) 

In [51]:
print(len(freq_response))
print(len(d2_list))
print(len(freq_list))
print(len(freq_response[0]))
print(len(freq_list[3]))

4
4
4
2531
3511
